# Ship
Reading in the ship MET and TSG data

## Load data

In [3]:
import glob
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.dates as mdates
from tqdm.notebook import tqdm_notebook as tqdm

def process_ctd_files(path_to_files, resample=True):
    """
    Process CTD files and create an xarray dataset.

    Parameters:
        path_to_files (str): Path to the CTD files.
        resample (bool): Flag indicating whether to resample the data to 10-minute intervals. Default is True.

    Returns:
        xr.Dataset: Processed CTD data as an xarray dataset.
    """
    tmp = []
    ctd_files = glob.glob(path_to_files)

    # Read and concatenate CTD data from multiple files
    print('Reading the files... 🕵🏼‍♀️')

    for i in tqdm(range(len(ctd_files)-1), desc="Processing CTD files"):
        df = pd.read_csv(ctd_files[i+1], skiprows=3, delim_whitespace=True)
        df['time'] = mdates.num2date([mdates.date2num(np.datetime64(f'2023-03-{str(5+i).zfill(2)}'))] * df.shape[0] + df.index.values / 86400)
        tmp.append(df)
    df = pd.concat(tmp)
    
    
    # Convert time to datetime, set as index, and create xarray dataset
    print('Converting dates... 📆')
    df['time'] = df['time'].dt.tz_localize(None).astype('datetime64[s]')
    df = df.set_index('time')
    ds = df.to_xarray()

    if resample:
        print('Resampling... 🫡')

        # Resample data to 10-minute intervals
        ds = ds.resample(time='10min').mean()

    # Calculate day of year, and mission day, and set them as coordinates
    ds['doy'] = xr.DataArray(mdates.date2num(ds['time']) - mdates.date2num(np.datetime64('2023-01-02')), dims='time')
    ds['mday'] = xr.DataArray(mdates.date2num(ds['time']) - mdates.date2num(ds['time'].values[0]), dims='time')
    ds = ds.set_coords('doy')
    ds = ds.set_coords('mday')

    print('Done 🥸')
    
    return ds

# Process CTD files and create xarray dataset
ds = process_ctd_files('/Volumes/MASSIVEUNIT/Data/QUICCHE/Ship drive copy/metacq/data/2303*.MET', resample=True)

Reading the files... 🕵🏼‍♀️


Processing CTD files:   0%|          | 0/26 [00:00<?, ?it/s]

Converting dates... 📆
Resampling... 🫡
Done 🥸


In [85]:
# Rename variables, otherwise saving to a netCDF will be difficult
rename_dict = {
    '#Time': 'Time',
    'RH-2': 'RH_2',
    'RT-2': 'RT_2',
    'DP-2': 'DP_2',
    'WS-2': 'WS_2',
    'WD-2': 'WD_2',
    'TW-2': 'TW_2',
    'TI-2': 'TI_2',
    'TT-2': 'TT_2',
    'TC-2': 'TC_2',
    'SA-2': 'SA_2',
    'SD-2': 'SD_2',
    'SV-2': 'SV_2',
    'TG-2': 'TG_2',
    'FI-2': 'FI_2',
    'FI-3': 'FI_3',
    'VP-2': 'VP_2',
    'VR-2': 'VR_2',
    'VH-2': 'VH_2',
    'ZO-2': 'ZO_2',
    'ZS-2': 'ZS_2',
    'ZT-2': 'ZT_2',
    'ZI-2': 'ZI_2',
    'TT-3': 'TT_3',
    'TC-3': 'TC_3',
    'SA-3': 'SA_3',
    'TT-4': 'TT_4',
    'TC-4': 'TC_4',
    'SA-4': 'SA_4'
}

ds = ds.rename(rename_dict)

### Add attributes

In [526]:
status_explanation = [
    "0: 'Data Valid - Calibration corrections not applied'",
    "1: 'Data Valid - Calibration corrections applied in Sensor module'",
    "2: 'Data Valid - Calibration corrections applied in Acquisition program'",
    "3: 'Data Valid - Both type 1 and type 2'",
    "7: 'Data Invalid - Out of range'",
    "8: 'Data Invalid - Data Emulated'",
    "9: 'Data Invalid - No Valid Response from Input'"
]

# Air Temperature Sensor
var = 'AT'
ds[var].attrs['module'] = 'TMP28'
ds[var].attrs['module_number'] = 1
ds[var].attrs['sensor'] = 'Air Temperature'
ds[var].attrs['model'] = '41342VC'
ds[var].attrs['serial'] = '27209'
ds[var].attrs['manufacturer'] = 'RM Young'
ds[var].attrs['location'] = "MET Mast 56' above MWL"
ds[var].attrs['installed_height'] = 17
ds[var].attrs['owner'] = 'SIO/STS'
ds[var].attrs['calibration_laboratory'] = 'STS'
ds[var].attrs['calibration_date'] = '20-Oct-2022'
ds[var].attrs['installation_date'] = '28-Oct-2022'
ds[var].attrs['comments'] = ''
ds[var].attrs['write_to_file'] = "True"
ds[var].attrs['number_parameters_returned_from_sensor'] = 1
ds[var].attrs['additional_parameters_computed'] = 0
ds[var].attrs['total'] = 1
ds[var].attrs['parameter_tags'] = 'AT'
ds[var].attrs['at_number'] = 1
ds[var].attrs['seconds_averaged'] = 60.0
ds[var].attrs['units'] = 'Deg C'
ds[var].attrs['long_name'] = 'Air Temperature'
ds[var].attrs['standard_name'] = 'air_temperature'
ds[var].attrs['valid_min'] = -40.0
ds[var].attrs['valid_max'] = 50.0
ds[var].attrs['status'] = '2-OK'
ds[var].attrs['status_explanation'] = status_explanation

# Relative Humidity Sensor
for i, var in enumerate(['RH', 'RT', 'DP', 'RH_2', 'RT_2', 'DP_2']):
    if var in ['RH', 'RT', 'DP']:
        ds[var].attrs['module'] = 'HRH06'
        ds[var].attrs['module_number'] = 1
    else:
        ds[var].attrs['module'] = 'HRH24'
        ds[var].attrs['module_number'] = 2

    ds[var].attrs['sensor'] = 'Relative Humidity'
    ds[var].attrs['model'] = 'EE08'
    ds[var].attrs['serial_number'] = '173305000106A2' if var in ['RH', 'RT', 'DP'] else '17150500023924'
    ds[var].attrs['manufacturer'] = 'E+E Elektronik'
    ds[var].attrs['location'] = "MET Mast 56'above MWL"
    ds[var].attrs['owner'] = 'SIO/STS'
    ds[var].attrs['calibration_lab'] = 'STS'
    ds[var].attrs['calibration_date'] = '25-Oct-2022'
    ds[var].attrs['installation_date'] = '28-Oct-2022'
    ds[var].attrs['comments'] = ''
    ds[var].attrs['write_to_file'] = "True"
    ds[var].attrs['number_parameters_returned_from_sensor'] = 2
    ds[var].attrs['additional_parameters_computed'] = 1
    ds[var].attrs['total'] = 3
    ds[var].attrs['parameter_tags'] = ['RH', 'RT', 'DP']
    ds[var].attrs[['rh_number', 'rt_number', 'dp_number'][i % 3]] = 1 if var in ['RH', 'RT', 'DP'] else 2
    ds[var].attrs['seconds_averaged'] = 60.0 if var in ['RH', 'RT'] else 0
    ds[var].attrs['units'] = 'percent' if var in ['RH', 'RH_2'] else 'degrees Celsius'
    ds[var].attrs['long_name'] = 'Relative Humidity' if var in ['RH', 'RH_2'] else 'Relative Temperature' if var in ['RT', 'RT_2'] else 'Dew-point Temperature'
    ds[var].attrs['standard_name'] = 'relative_humidity' if var in ['RH', 'RH_2'] else 'air_temperature' if var in ['RT', 'RT_2'] else 'dew_point_temperature'
    ds[var].attrs['valid_min'] = 0.0 if var in ['RH', 'RH_2'] else -40.0
    ds[var].attrs['valid_max'] = 100.0 if var in ['RH', 'RH_2'] else 60.0
    ds[var].attrs['status'] = '2-OK'
    ds[var].attrs['status_explanation'] = status_explanation
    ds[var].attrs['installed_height'] = 17

In [527]:
# Wind Sensors

for i, var in enumerate(['WS', 'WD', 'TW', 'TI', 'WS_2', 'WD_2', 'TW_2', 'TI_2']):
    if var in ['WS', 'WD', 'TW', 'TI']:
        ds[var].attrs['module'] = 'WND97'
        ds[var].attrs['module_number'] = 1
    else:
        ds[var].attrs['module'] = 'WND09'
        ds[var].attrs['module_number'] = 2

    ds[var].attrs['sensor'] = 'Wind'
    ds[var].attrs['model'] = '86106' if var in ['WS', 'WD', 'TW', 'TI'] else '5300'
    ds[var].attrs['serial_number'] = '1297' if var in ['WS', 'WD', 'TW', 'TI'] else '17150500023924'
    ds[var].attrs['manufacturer'] = 'RM Young'
    ds[var].attrs['location'] = "MET Mast 56'above MWL" if var in ['WS', 'WD', 'TW', 'TI'] else "Main Mast Starboard Side 82' above MWL"
    ds[var].attrs['owner'] = 'SIO/STS' if var in ['WS', 'WD', 'TW', 'TI'] else 'RV Roger Revelle'
    ds[var].attrs['calibration_lab'] = 'STS' if var in ['WS', 'WD', 'TW', 'TI'] else 'RM Young'
    ds[var].attrs['calibration_date'] = '11-Oct-2022' if var in ['WS', 'WD', 'TW', 'TI'] else '????'
    ds[var].attrs['installation_date'] = '28-Oct-2022' if var in ['WS', 'WD', 'TW', 'TI'] else '1996'
    ds[var].attrs['comments'] = ''
    ds[var].attrs['write_to_file'] = "True"
    ds[var].attrs['number_parameters_returned_from_sensor'] = 2
    ds[var].attrs['additional_parameters_computed'] = 2
    ds[var].attrs['total'] = 4
    ds[var].attrs['parameter_tags'] = ['WS', 'WD', 'TW', 'TI']
    ds[var].attrs[['ws_number', 'wd_number', 'tw_number', 'ti_number'][i % 4]] = 1 if var in ['WS', 'WD', 'TW', 'TI'] else 2
    ds[var].attrs['status'] = '2-OK'
    ds[var].attrs['seconds_averaged'] = 60.0 if var in ['WS', 'WD'] else 0
    ds[var].attrs['units'] = 'm/s' if var in ['WS', 'WS_2', 'TW', 'TW_2'] else 'degrees'
    ds[var].attrs['long_name'] = 'Relative Wind Speed' if var in ['WS', 'WS_2'] else 'Relative Wind Direction' if var in ['WD', 'WD_2'] else 'True Wind Speed' if var in ['TW', 'TW_2'] else 'True Wind Direction'
    ds[var].attrs['standard_name'] = 'relative_wind_speed' if var in ['WS', 'WS_2'] else 'relative_wind_direction' if var in ['WD', 'WD_2'] else 'true_wind_peed' if var in ['TW', 'TW_2'] else 'true_wind_direction'
    ds[var].attrs['valid_min'] = 0.0
    ds[var].attrs['valid_max'] = 500.0 if var in ['WS', 'WS_2', 'TW', 'TW_2'] else 360.0
    ds[var].attrs['status'] = '2-OK'
    ds[var].attrs['status_explanation'] = status_explanation
    ds[var].attrs['installed_height'] = 17 if var in ['WS', 'WD', 'TW', 'TI'] else 25

In [528]:
# Thermosalinograph Sensors

for i, var in enumerate(['TT', 'TC', 'SA', 'SD', 'SV', 'TG', 'TT_2', 'TC_2', 'SA_2', 'SD_2', 'SV_2', 'TG_2']):
    if var in ['TT', 'TC', 'SA', 'SD', 'SV', 'TG']:
        ds[var].attrs['module'] = 'TSG82'
        ds[var].attrs['module_number'] = 1
    else:
        ds[var].attrs['module'] = 'TSG18'
        ds[var].attrs['module_number'] = 2

    ds[var].attrs['sensor'] = 'Thermosalinograph'
    ds[var].attrs['model'] = 'SBE45'
    ds[var].attrs['serial_number'] = '0082' if var in ['TT', 'TC', 'SA', 'SD', 'SV', 'TG'] else '0318'
    ds[var].attrs['manufacturer'] = 'Seabird'
    ds[var].attrs['location'] = 'Bow thruster UCW intake' if var in ['TT', 'TC', 'SA', 'SD', 'SV', 'TG'] else 'Aft Hydro Lab'
    ds[var].attrs['owner'] = 'SIO/STS'
    ds[var].attrs['calibration_lab'] = 'Seabird'
    ds[var].attrs['calibration_date'] = '14-Jul-2022'
    ds[var].attrs['installation_date'] = '30-Oct-2022'
    ds[var].attrs['comments'] = ''
    ds[var].attrs['write_to_file'] = 'True'
    ds[var].attrs['number_parameters_returned_from_sensor'] = 2
    ds[var].attrs['additional_parameters_computed'] = 4
    ds[var].attrs['total'] = 6
    ds[var].attrs['parameter_tags'] = ['TT', 'TC', 'SA', 'SD', 'SV', 'TG']
    ds[var].attrs[['tt_number', 'tc_number', 'sa_number', 'sd_number', 'sv_number', 'tg_number'][i % 6]] = 1 if var in ['TT', 'TC', 'SA', 'SD', 'SV', 'TG'] else 2
    ds[var].attrs['status'] = '2-OK'
    ds[var].attrs['seconds_averaged'] = 60.0 if var in ['TT', 'TC'] else 0
    ds[var].attrs['units'] = 'Deg C' if var in ['TT', 'TT_2'] else 'mS/cm' if var in ['TC', 'TC_2'] else 'PSU' if var in ['SA', 'SA_2'] else 'Kg/m^3' if var in ['SD', 'SD_2'] else 'm/s' if var in ['SV', 'SV_2'] else ''
    ds[var].attrs['long_name'] = 'Thermosalinograph Temperature' if var in ['TT', 'TT_2'] else 'Thermosalinograph Conductivity' if var in ['TC', 'TC_2'] else 'Salinity' if var in ['SA', 'SA_2'] else 'Sigma_t' if var in ['SD', 'SD_2'] else 'Sound Velocity' if var in ['SV', 'SV_2'] else 'Thermosalinograph Conductivity'
    ds[var].attrs['standard_name'] = 'sea_water_temperature' if var in ['TT', 'TT_2'] else 'sea_water_electrical_conductivity' if var in ['TC', 'TC_2'] else 'sea_water_salinity' if var in ['SA', 'SA_2'] else 'sea_water_density' if var in ['SD', 'SD_2'] else 'sound_velocity' if var in ['SV', 'SV_2'] else 'sea_water_electrical_conductivity'
    ds[var].attrs['valid_min'] = -5.0e0 if var in ['TT', 'TT_2'] else -5.0e0 if var in ['TC', 'TC_2'] else 0.0e0 if var in ['SA', 'SA_2'] else -1.0e1 if var in ['SD', 'SD_2'] else 1.0e3 if var in ['SV', 'SV_2'] else -1.0e10 if var in ['TG', 'TG_2'] else ''
    ds[var].attrs['valid_max'] = 5.0e1 if var in ['TT', 'TT_2'] else 1.0e5 if var in ['TC', 'TC_2'] else 1.0e2 if var in ['SA', 'SA_2'] else 7.0e3 if var in ['SD', 'SD_2'] else 2.0e3 if var in ['SV', 'SV_2'] else 1.0e10 if var in ['TG', 'TG_2'] else ''
    ds[var].attrs['status_explanation'] = status_explanation
    ds[var].attrs['installed_height'] = "????"

In [529]:
# Surface Temperature Sensor
var = 'ST'
ds[var].attrs['module'] = 'SST53'
ds[var].attrs['module_number'] = 1
ds[var].attrs['sensor'] = 'Sea Surface Temperature'
ds[var].attrs['model'] = 'SBE38'
ds[var].attrs['serial'] = '1188'
ds[var].attrs['manufacturer'] = 'Seabird'
ds[var].attrs['location'] = "Bow thruster UCW intake"
ds[var].attrs['installed_height'] = "????"
ds[var].attrs['owner'] = 'SIO/STS'
ds[var].attrs['calibration_laboratory'] = 'STS'
ds[var].attrs['calibration_date'] = '05-Jan-2021'
ds[var].attrs['installation_date'] = '27-May-2021'
ds[var].attrs['comments'] = ''
ds[var].attrs['write_to_file'] = "True"
ds[var].attrs['number_parameters_returned_from_sensor'] = 1
ds[var].attrs['additional_parameters_computed'] = 0
ds[var].attrs['total'] = 1
ds[var].attrs['parameter_tags'] = 'ST'
ds[var].attrs['st_number'] = 1
ds[var].attrs['seconds_averaged'] = 60.0
ds[var].attrs['units'] = 'Deg C'
ds[var].attrs['long_name'] = 'Sea Surface Temperature'
ds[var].attrs['standard_name'] = 'sea_water_temperature'
ds[var].attrs['valid_min'] = -5.0
ds[var].attrs['valid_max'] = 50.0
ds[var].attrs['status'] = '1-OK'
ds[var].attrs['status_explanation'] = status_explanation

In [530]:
# Attitude sensor (for heading)
for i, var in enumerate(['SH', 'SM', 'SR']):
    
    ds[var].attrs['module'] = 'ASH01'
    ds[var].attrs['module_number'] = 1
    ds[var].attrs['sensor'] = 'Attitude'
    ds[var].attrs['model'] = 'ABX-Two'
    ds[var].attrs['serial_number'] = '544C02743'
    ds[var].attrs['manufacturer'] = 'Trimble'
    ds[var].attrs['location'] = "Bridge starboard"
    ds[var].attrs['owner'] = 'SIO/STS'
    ds[var].attrs['calibration_lab'] = 'n/a'
    ds[var].attrs['calibration_date'] = '24-Oct-2020'
    ds[var].attrs['installation_date'] = '15-Oct-2020'
    ds[var].attrs['comments'] = ''
    ds[var].attrs['write_to_file'] = "True"
    ds[var].attrs['number_parameters_returned_from_sensor'] = 3
    ds[var].attrs['additional_parameters_computed'] = 0
    ds[var].attrs['total'] = 3
    ds[var].attrs['parameter_tags'] = ['SH', 'SM', 'SR']
    ds[var].attrs[['sh_number', 'sm_number', 'sr_number'][i % 3]] = 1
    ds[var].attrs['seconds_averaged'] = 0.0
    ds[var].attrs['units'] = 'degrees'
    ds[var].attrs['long_name'] = 'Heading' if var in ['SH'] else 'Pitch' if var in ['SM'] else 'Roll'
    ds[var].attrs['standard_name'] = 'Heading' if var in ['SH'] else 'pitch' if var in ['SM'] else 'poll'
    ds[var].attrs['valid_min'] = 0 if var in ['SH'] else -1e10
    ds[var].attrs['valid_max'] = 360 if var in ['SH'] else 1e10
    ds[var].attrs['status'] = '1-OK'
    ds[var].attrs['status_explanation'] = status_explanation

In [531]:
# Longwave radiation sensor
for i, var in enumerate(['LD', 'LB', 'LT', 'LW']):
    
    ds[var].attrs['module'] = 'LWR21'
    ds[var].attrs['module_number'] = 1
    ds[var].attrs['sensor'] = 'Long Wave Radiation'
    ds[var].attrs['model'] = 'PIR'
    ds[var].attrs['serial_number'] = '38301F3'
    ds[var].attrs['manufacturer'] = 'Eppley Labs'
    ds[var].attrs['location'] = "MET Mast 68'above MWL"
    ds[var].attrs['owner'] = 'SIO/STS'
    ds[var].attrs['calibration_lab'] = 'Eppley'
    ds[var].attrs['calibration_date'] = '05-Oct-2022'
    ds[var].attrs['installation_date'] = '27-Oct-2022'
    ds[var].attrs['comments'] = ''
    ds[var].attrs['write_to_file'] = "True"
    ds[var].attrs['number_parameters_returned_from_sensor'] = 3
    ds[var].attrs['additional_parameters_computed'] = 1
    ds[var].attrs['total'] = 4
    ds[var].attrs['parameter_tags'] = ['LD', 'LB', 'LT', 'LW']
    ds[var].attrs[['ld_number', 'lb_number', 'lt_number', 'lw_number'][i % 4]] = 1
    ds[var].attrs['seconds_averaged'] = 0.0
    ds[var].attrs['units'] = 'degrees' if var in ['LD', 'LB'] else 'uv' if var in ['LT'] else 'W/m2'
    ds[var].attrs['long_name'] = 'Long Wave Radiation Dome Temperature' if var in ['LD'] else 'Long Wave Radiation Body Temperature' if var in ['LB'] else 'Long Wave Radiation Thermopile Volts' if var in ['LT'] else 'Long Wave Radiation'
    ds[var].attrs['standard_name'] = 'longwave_radiation_dome_temperature' if var in ['LD'] else 'longwave_radiation_body_temperature' if var in ['LB'] else 'longwave_radiation_thermopile_volts' if var in ['LT'] else 'longwave_radiation'
    ds[var].attrs['valid_min'] = -1e10 if var in ['LD', 'LB'] else -1e5
    ds[var].attrs['valid_max'] = 1e5
    ds[var].attrs['status'] = '1-OK'
    ds[var].attrs['status_explanation'] = status_explanation

In [532]:
# Shortwave radiation sensor
var = 'SW'

ds[var].attrs['module'] = 'SWR21'
ds[var].attrs['module_number'] = 1
ds[var].attrs['sensor'] = 'Short Wave Radiation'
ds[var].attrs['model'] = 'SPP'
ds[var].attrs['serial_number'] = '38361F3'
ds[var].attrs['manufacturer'] = 'Eppley Labs'
ds[var].attrs['location'] = "MET Mast 68'above MWL"
ds[var].attrs['owner'] = 'SIO/STS'
ds[var].attrs['calibration_lab'] = 'Eppley Labs'
ds[var].attrs['calibration_date'] = '03-Oct-2022'
ds[var].attrs['installation_date'] = '27-Oct-2022'
ds[var].attrs['comments'] = ''
ds[var].attrs['write_to_file'] = "True"
ds[var].attrs['number_parameters_returned_from_sensor'] = 1
ds[var].attrs['additional_parameters_computed'] = 0
ds[var].attrs['total'] = 1
ds[var].attrs['parameter_tags'] = ['SW']
ds[var].attrs['sw_number'] = 1
ds[var].attrs['seconds_averaged'] = 0.0
ds[var].attrs['units'] = 'W/m2'
ds[var].attrs['long_name'] = 'Shortwave Radiation'
ds[var].attrs['standard_name'] = 'shortwave_radiation'
ds[var].attrs['valid_min'] = -5.0e1
ds[var].attrs['valid_max'] = 5.0e4
ds[var].attrs['status'] = '2-OK'
ds[var].attrs['status_explanation'] = status_explanation

In [ ]:
# Save the dataset
ds.to_netcdf('/Users/xedhjo/Documents/Projects/QUICCHE/data/met_data.nc')